In [2]:
import numpy as np
import pandas as pd
import re

In [3]:
PATH_tweets = '../Data/tweets.csv'

In [4]:
tweets = pd.read_csv(
    PATH_tweets,
    usecols=['id', 'screen_name', 'text'], 
    index_col='id', 
    dtype={'screen_name': str, 'text': str}
    )
tweets = tweets.drop_duplicates(subset='text', keep=False)
tweets = tweets.dropna()

In [5]:
def limpiar_retweet(tweet:str):
    if tweet.startswith('RT'):
        tweet = ''.join(tweet.split(': ')[1:])
    return tweet

def limpiar_mencion(tweet:str):
    tweet = re.sub(r'@\w+', '', tweet)
    return tweet

tweets['text'] = tweets['text'].apply(limpiar_retweet)
tweets['text'] = tweets['text'].apply(limpiar_mencion)

In [6]:
tweets.loc[:, 'text'] = tweets['text'].str\
        .replace('[.,]', '', regex=True).str.lower()\
        .apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))

In [7]:
# Save tweets
tweets.to_parquet('../Data/tweets.parquet')

In [8]:
def shingles(k, words: np.ndarray):
    shingles = []
    for i in range(0, len(words)):
        shingles.append(' '.join(words[i:i+k]))
    return list(set(shingles))

In [16]:
k = 1 # Shingles size
tweets['shingles'] = tweets['text'].str.split().apply(lambda x: np.array(x)).apply(lambda x: shingles(k, x))

In [17]:
from sklearn.preprocessing import MultiLabelBinarizer

In [18]:
shings = tweets['shingles'].to_numpy()
mlb = MultiLabelBinarizer(sparse_output=True)
caracteristica = mlb.fit_transform(shings)

In [19]:
# !pip install pyarrow -q

In [20]:
# Save csr matrix
from scipy.sparse import save_npz

# k 2 por los shingles
save_npz('../Data/caracteristica_k2.npz', caracteristica)